In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, f1_score
from tensorflow.keras.callbacks import TensorBoard, EarlyStopping
import time

# CBAM (Convolutional Block Attention Module) Layer
class CBAM(layers.Layer):
    def __init__(self):
        super(CBAM, self).__init__()
        self.channel_attention = ChannelAttention()
        self.spatial_attention = SpatialAttention()
    
    def call(self, inputs):
        channel_att = self.channel_attention(inputs)
        spatial_att = self.spatial_attention(inputs)
        return tf.multiply(inputs, tf.expand_dims(tf.expand_dims(channel_att, axis=1), axis=1)) * spatial_att

# Channel Attention Layer
class ChannelAttention(layers.Layer):
    def __init__(self):
        super(ChannelAttention, self).__init__()
        self.global_avgpool = layers.GlobalAveragePooling2D()
        self.dense1 = layers.Dense(64, activation='relu')
        self.dense2 = layers.Dense(1, activation='sigmoid')
    
    def call(self, inputs):
        avg_pool = self.global_avgpool(inputs)
        dense1_out = self.dense1(avg_pool)
        attention = self.dense2(dense1_out)
        return attention

# Linear Attention Layer
class LinearAttention(layers.Layer):
    def __init__(self):
        super(LinearAttention, self).__init__()
        self.dense = layers.Dense(1)
    
    def call(self, inputs):
        return tf.nn.sigmoid(self.dense(inputs))

# NonLinear Attention Layer
class NonLinearAttention(layers.Layer):
    def __init__(self):
        super(NonLinearAttention, self).__init__()
        self.dense1 = layers.Dense(64, activation='relu')
        self.dense2 = layers.Dense(1, activation='sigmoid')
    
    def call(self, inputs):
        dense1_out = self.dense1(inputs)
        attention = self.dense2(dense1_out)
        return attention

def configure_model(layers_config, optimizer='adam', loss='mean_squared_error', metrics=['accuracy'],debugflag=False):
    if debugflag:
        model = tf.keras.Sequential([
            layers.LSTM(32, activation='relu', input_shape=(2,1)),
            layers.Dense(64, activation='relu'),
            layers.Dense(1, activation='linear')  # Output layer with linear activation for regression
            ])
        model.compile(optimizer=optimizer, loss=loss, metrics=metrics)
        return model
        
    model = tf.keras.Sequential()

    for layer_config in layers_config:
        layer_type = layer_config[0]

        if layer_type == 'dense':
            neurons, activation = layer_config[1], layer_config[2]
            if len(layer_config) > 3 and layer_config[3] == 'input':
                input_shape = layer_config[4]
                model.add(layers.Dense(neurons, activation=activation, input_shape=(input_shape,)))
            else:
                model.add(layers.Dense(neurons, activation=activation))
        elif layer_type == 'dropout':
            rate = layer_config[1]
            model.add(layers.Dropout(rate))
        elif layer_type == 'activation':
            activation = layer_config[1]
            model.add(layers.Activation(activation))
        elif layer_type == 'conv2d':
            filters, kernel_size, strides, activation = layer_config[1], layer_config[2], layer_config[3], layer_config[4]
            model.add(layers.Conv2D(filters, kernel_size=kernel_size, strides=strides, activation=activation))
        elif layer_type == 'maxpool2d':
            pool_size = layer_config[1]
            model.add(layers.MaxPool2D(pool_size=pool_size))
        elif layer_type == 'rnn':
            units, return_sequences = layer_config[1], layer_config[2]
            model.add(layers.SimpleRNN(units, return_sequences=return_sequences))
        elif layer_type == 'lstm':
            units, return_sequences = layer_config[1], layer_config[2]
            model.add(layers.LSTM(units, return_sequences=return_sequences))
        elif layer_type == 'gru':
            units, return_sequences = layer_config[1], layer_config[2]
            model.add(layers.GRU(units, return_sequences=return_sequences))
        elif layer_type == 'attention':
            model.add(layers.MultiHeadAttention(num_heads=layer_config[1], key_dim=layer_config[2]))
        elif layer_type == 'cbam':
            model.add(CBAM())
        elif layer_type == 'channel_attention':
            model.add(ChannelAttention())
        elif layer_type == 'linear_attention':
            model.add(LinearAttention())
        elif layer_type == 'nonlinear_attention':
            model.add(NonLinearAttention())

    model.compile(optimizer=optimizer, loss=loss, metrics=metrics)

    return model


def train_model(model, X_train, y_train, X_test, y_test, model_name='perceptron', dataset_name="addition_dataset_2", total_epoch=50,ea=False,eamonitor="val_loss"):
    tensorboard_callback = TensorBoard(log_dir='./../../Observation/'+model_name+'/'+model_name+dataset_name+'logs')
    # Define EarlyStopping callback
    early_stopping = EarlyStopping(monitor=eamonitor, patience=10, min_delta=0.001, verbose=1)


    start_time = time.time()
    if ea:
        history = model.fit(X_train, y_train, epochs=total_epoch, validation_data=(X_test, y_test), callbacks=[tensorboard_callback,early_stopping])
    else:
        history = model.fit(X_train, y_train, epochs=total_epoch, validation_data=(X_test, y_test), callbacks=[tensorboard_callback])
    training_time = time.time() - start_time

    predictions = model.predict(X_test)

    return model, history, predictions, training_time

def test_model(model, X_test, y_test, model_name='perceptron', dataset_name="addition_dataset_2",threshold=0.5):
    start_time = time.time()
    predictions = model.predict(X_test)
    inference_time = time.time() - start_time
    
    metrics = calculate_metrics(predictions, y_test,threshold)
    metrics['Training Time'] = training_time
    metrics['Inference Time'] = inference_time
    metrics['Model type'] = model_name
    metrics['dataset'] = dataset_name

    print("Metrics:")
    for metric, value in metrics.items():
        print(f"{metric}: {value}")

    metrics_list = list(metrics.values())
    pd.DataFrame([metrics_list], columns=metrics.keys()).to_csv('./../../Observation/'+model_name+'/'+model_name+dataset_name+'metrics.csv', index=False)

    return metrics

def calculate_metrics(predictions, true_labels, threshold=0.5):
    #threshold = 0.5  # Define the threshold
    interpredictions=predictions.flatten()
    correct_predictions = np.sum(np.abs(true_labels - interpredictions) <= threshold)
    total_predictions = len(true_labels)
    accuracy = correct_predictions / total_predictions  # Final accuracy
    mse = mean_squared_error(true_labels, interpredictions)  # Mean Squared Error
    mae = mean_absolute_error(true_labels, interpredictions)  # Mean Absolute Error
    rmse = np.sqrt(mse)  # Root Mean Squared Error
    f1 = f1_score(true_labels.round(), interpredictions.round(), average='micro')  # F1 Score

    return {
        'Accuracy': accuracy,
        'MSE': mse,
        'MAE': mae,
        'RMSE': rmse,
        'F1 Score': f1
    }


In [2]:
# Load dataset
dataset_name = "addition_dataset_2"
d_set = pd.read_csv('./../../Data/'+dataset_name+'.csv')
dset_features = d_set.copy()
dset_labels = dset_features.pop('result')
dset_features = np.array(dset_features)

# Reshape features to include timestep dimension
# Assuming each row in the CSV file represents a timestep
# Reshape to (number_of_samples, timesteps, number_of_features)
timesteps = 1  # Assuming each row represents a timestep
dset_features_reshaped = dset_features.reshape(-1, timesteps, dset_features.shape[1])

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(dset_features, dset_labels, test_size=0.2, random_state=42)

# Configure the model with multiple layers
model = tf.keras.Sequential([
            layers.LSTM(32, activation='relu', input_shape=(dset_features.shape[1],1)),
            layers.Dense(64, activation='relu'),
            layers.Dense(1, activation='linear')  # Output layer with linear activation for regression
            ])
model.compile(optimizer="adam", loss="mean_squared_error", metrics=['mae'])

# Train the model
model, history, predictions, training_time = train_model(model, X_train, y_train, X_test, y_test,model_name='LSTM',dataset_name=dataset_name,total_epoch=50)

# Test the model
test_metrics = test_model(model, X_test, y_test,model_name='LSTM',dataset_name=dataset_name)

c:\Users\akars\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 145776208.0000 - mae: 6698.7310 - val_loss: 0.3486 - val_mae: 0.5007
Epoch 2/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.3482 - mae: 0.4960 - val_loss: 0.3395 - val_mae: 0.4895
Epoch 3/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.3346 - mae: 0.4863 - val_loss: 0.3356 - val_mae: 0.4969
Epoch 4/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.3351 - mae: 0.4858 - val_loss: 0.2931 - val_mae: 0.4551
Epoch 5/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.2966 - mae: 0.4548 - val_loss: 0.2898 - val_mae: 0.4650
Epoch 6/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.3290 - mae: 0.4659 - val_loss: 0.3042 - val_mae: 0.4529
Epoch 7/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.2788 - mae: 0.4452 - val_loss: 0.5898 - val_mae: 0.6122
Epoch 8/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.3246 - mae: 0.4707 - val_loss: 0.3308 - val_mae: 0.4755
Epoch 9/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1m

In [3]:
# Load dataset
dataset_name = "doubling_dataset"
d_set = pd.read_csv('./../../Data/'+dataset_name+'.csv')
dset_features = d_set.copy()
dset_labels = dset_features.pop('result')
dset_features = np.array(dset_features)

# Reshape features to include timestep dimension
# Assuming each row in the CSV file represents a timestep
# Reshape to (number_of_samples, timesteps, number_of_features)
timesteps = 1  # Assuming each row represents a timestep
dset_features_reshaped = dset_features.reshape(-1, timesteps, dset_features.shape[1])

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(dset_features, dset_labels, test_size=0.2, random_state=42)

# Configure the model with multiple layers
model = tf.keras.Sequential([
            layers.LSTM(32, activation='relu', input_shape=(dset_features.shape[1],1)),
            layers.Dense(64, activation='relu'),
            layers.Dense(1, activation='linear')  # Output layer with linear activation for regression
            ])
model.compile(optimizer="adam", loss="mean_squared_error", metrics=['mae'])

# Train the model
model, history, predictions, training_time = train_model(model, X_train, y_train, X_test, y_test,model_name='LSTM',dataset_name=dataset_name,total_epoch=50)

# Test the model
test_metrics = test_model(model, X_test, y_test,model_name='LSTM',dataset_name=dataset_name)

Epoch 1/50


c:\Users\akars\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


500/500 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - loss: 207082832.0000 - mae: 9229.3076 - val_loss: 1.0807 - val_mae: 0.8714
Epoch 2/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 1.1068 - mae: 0.8839 - val_loss: 1.0935 - val_mae: 0.8784
Epoch 3/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 1.1032 - mae: 0.8826 - val_loss: 1.1054 - val_mae: 0.8771
Epoch 4/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 1.1063 - mae: 0.8804 - val_loss: 1.0660 - val_mae: 0.8672
Epoch 5/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 1.0714 - mae: 0.8675 - val_loss: 1.0580 - val_mae: 0.8644
Epoch 6/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 1.0527 - mae: 0.8547 - val_loss: 1.0443 - val_mae: 0.8576
Epoch 7/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 1.0340 - mae: 0.8488 - val_loss: 1.0710 - val_mae: 0.8544
Epoch 8/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 1.0817 - mae: 0.8658 - val_loss: 1.0153 - val_mae: 0.8498
Epoch 9/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - lo

In [5]:
# Load dataset
dataset_name = "substraction_dataset"
d_set = pd.read_csv('./../../Data/'+dataset_name+'.csv')
dset_features = d_set.copy()
dset_labels = dset_features.pop('result')
dset_features = np.array(dset_features)

# Reshape features to include timestep dimension
# Assuming each row in the CSV file represents a timestep
# Reshape to (number_of_samples, timesteps, number_of_features)
timesteps = 1  # Assuming each row represents a timestep
dset_features_reshaped = dset_features.reshape(-1, timesteps, dset_features.shape[1])

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(dset_features, dset_labels, test_size=0.2, random_state=42)

# Configure the model with multiple layers
model = tf.keras.Sequential([
            layers.LSTM(32, activation='relu', input_shape=(dset_features.shape[1],1)),
            layers.Dense(64, activation='relu'),
            layers.Dense(1, activation='linear')  # Output layer with linear activation for regression
            ])
model.compile(optimizer="adam", loss="mean_squared_error", metrics=['mae'])

# Train the model
model, history, predictions, training_time = train_model(model, X_train, y_train, X_test, y_test,model_name='LSTM',dataset_name=dataset_name,total_epoch=50)

# Test the model
test_metrics = test_model(model, X_test, y_test,model_name='LSTM',dataset_name=dataset_name)

Epoch 1/50


c:\Users\akars\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


500/500 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 11654741.0000 - mae: 1894.4353 - val_loss: 0.0546 - val_mae: 0.1942
Epoch 2/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.0552 - mae: 0.1946 - val_loss: 0.0540 - val_mae: 0.1953
Epoch 3/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.0548 - mae: 0.1954 - val_loss: 0.0600 - val_mae: 0.1984
Epoch 4/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.0547 - mae: 0.1953 - val_loss: 0.0527 - val_mae: 0.1912
Epoch 5/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.0534 - mae: 0.1928 - val_loss: 0.0521 - val_mae: 0.1928
Epoch 6/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 2.0750 - mae: 0.3668 - val_loss: 0.0561 - val_mae: 0.1944
Epoch 7/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.0533 - mae: 0.1913 - val_loss: 0.0562 - val_mae: 0.1975
Epoch 8/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.0528 - mae: 0.1915 - val_loss: 0.0500 - val_mae: 0.1884
Epoch 9/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - los

In [5]:
# Load dataset
dataset_name = "simple_sin_dataset"
d_set = pd.read_csv('./../../Data/'+dataset_name+'.csv')
dset_features = d_set.copy()
dset_labels = dset_features.pop('result')
dset_features = np.array(dset_features)

# Reshape features to include timestep dimension
# Assuming each row in the CSV file represents a timestep
# Reshape to (number_of_samples, timesteps, number_of_features)
timesteps = 1  # Assuming each row represents a timestep
dset_features_reshaped = dset_features.reshape(-1, timesteps, dset_features.shape[1])

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(dset_features, dset_labels, test_size=0.2, random_state=42)

# Configure the model with multiple layers
model = tf.keras.Sequential([
            layers.LSTM(32, activation='relu', input_shape=(dset_features.shape[1],1)),
            layers.Dense(64, activation='relu'),
            layers.Dense(1, activation='linear')  # Output layer with linear activation for regression
            ])
model.compile(optimizer="adam", loss="mean_squared_error", metrics=['mae'])

# Train the model
model, history, predictions, training_time = train_model(model, X_train, y_train, X_test, y_test,model_name='LSTM',dataset_name=dataset_name,total_epoch=50)

# Test the model
test_metrics = test_model(model, X_test, y_test,model_name='LSTM',dataset_name=dataset_name)

Epoch 1/50


c:\Users\akars\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


500/500 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - loss: 43651.4844 - mae: 73.0236 - val_loss: 0.5087 - val_mae: 0.6441
Epoch 2/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.5018 - mae: 0.6334 - val_loss: 0.5178 - val_mae: 0.6469
Epoch 3/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.5130 - mae: 0.6408 - val_loss: 0.5227 - val_mae: 0.6474
Epoch 4/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.5153 - mae: 0.6406 - val_loss: 0.5193 - val_mae: 0.6464
Epoch 5/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.5205 - mae: 0.6442 - val_loss: 0.5211 - val_mae: 0.6479
Epoch 6/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.5261 - mae: 0.6465 - val_loss: 0.5138 - val_mae: 0.6455
Epoch 7/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.5354 - mae: 0.6469 - val_loss: 0.5477 - val_mae: 0.6560
Epoch 8/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.5276 - mae: 0.6412 - val_loss: 0.5357 - val_mae: 0.6509
Epoch 9/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.

In [7]:
# Load dataset
dataset_name = "simple_cos_dataset"
d_set = pd.read_csv('./../../Data/'+dataset_name+'.csv')
dset_features = d_set.copy()
dset_labels = dset_features.pop('result')
dset_features = np.array(dset_features)

# Reshape features to include timestep dimension
# Assuming each row in the CSV file represents a timestep
# Reshape to (number_of_samples, timesteps, number_of_features)
timesteps = 1  # Assuming each row represents a timestep
dset_features_reshaped = dset_features.reshape(-1, timesteps, dset_features.shape[1])

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(dset_features, dset_labels, test_size=0.2, random_state=42)

# Configure the model with multiple layers
model = tf.keras.Sequential([
            layers.LSTM(32, activation='relu', input_shape=(dset_features.shape[1],1)),
            layers.Dense(64, activation='relu'),
            layers.Dense(1, activation='linear')  # Output layer with linear activation for regression
            ])
model.compile(optimizer="adam", loss="mean_squared_error", metrics=['mae'])

# Train the model
model, history, predictions, training_time = train_model(model, X_train, y_train, X_test, y_test,model_name='LSTM',dataset_name=dataset_name,total_epoch=50)

# Test the model
test_metrics = test_model(model, X_test, y_test,model_name='LSTM',dataset_name=dataset_name)

Epoch 1/50


c:\Users\akars\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


500/500 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - loss: 395.2218 - mae: 7.2930 - val_loss: 0.4958 - val_mae: 0.6339
Epoch 2/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.5096 - mae: 0.6426 - val_loss: 0.5038 - val_mae: 0.6359
Epoch 3/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.5079 - mae: 0.6405 - val_loss: 0.4985 - val_mae: 0.6345
Epoch 4/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.5060 - mae: 0.6378 - val_loss: 0.4957 - val_mae: 0.6341
Epoch 5/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.5153 - mae: 0.6441 - val_loss: 0.5012 - val_mae: 0.6352
Epoch 6/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.5066 - mae: 0.6361 - val_loss: 0.5122 - val_mae: 0.6397
Epoch 7/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.5241 - mae: 0.6478 - val_loss: 0.5049 - val_mae: 0.6373
Epoch 8/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.5218 - mae: 0.6429 - val_loss: 0.5279 - val_mae: 0.6430
Epoch 9/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.544

In [10]:
# Load dataset
dataset_name = "multiplication_dataset"
d_set = pd.read_csv('./../../Data/'+dataset_name+'.csv')
dset_features = d_set.copy()
dset_labels = dset_features.pop('result')
dset_features = np.array(dset_features)

# Reshape features to include timestep dimension
# Assuming each row in the CSV file represents a timestep
# Reshape to (number_of_samples, timesteps, number_of_features)
timesteps = 1  # Assuming each row represents a timestep
dset_features_reshaped = dset_features.reshape(-1, timesteps, dset_features.shape[1])

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(dset_features, dset_labels, test_size=0.2, random_state=42)

# Configure the model with multiple layers
model = tf.keras.Sequential([
            layers.LSTM(32, activation='relu', input_shape=(dset_features.shape[1],1)),
            layers.Dense(64, activation='relu'),
            layers.Dense(1, activation='linear')  # Output layer with linear activation for regression
            ])
model.compile(optimizer="adam", loss="mean_squared_error", metrics=['mae'])

# Train the model
model, history, predictions, training_time = train_model(model, X_train, y_train, X_test, y_test,model_name='LSTM',dataset_name=dataset_name,total_epoch=1500,ea=True,eamonitor="mae")

# Test the model
test_metrics = test_model(model, X_test, y_test,model_name='LSTM',dataset_name=dataset_name)

Epoch 1/1500


c:\Users\akars\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


500/500 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 31551007997558784.0000 - mae: 132155344.0000 - val_loss: 28591324559048704.0000 - val_mae: 124804800.0000
Epoch 2/1500
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 29484351339102208.0000 - mae: 127265080.0000 - val_loss: 22062714423607296.0000 - val_mae: 106468944.0000
Epoch 3/1500
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 21976340483801088.0000 - mae: 105173248.0000 - val_loss: 29012478314676224.0000 - val_mae: 125920880.0000
Epoch 4/1500
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 29227523032219648.0000 - mae: 126513016.0000 - val_loss: 26920880781131776.0000 - val_mae: 120306512.0000
Epoch 5/1500
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 27198793082470400.0000 - mae: 121129912.0000 - val_loss: 23521901645135872.0000 - val_mae: 110743336.0000
Epoch 6/1500
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 22718386278498304.0000 - mae: 108329616.0000 - val_loss: 18499485000794112.0000 - val_mae: 95602360.0000
Epoch 

In [17]:
# Load dataset
dataset_name = "exponent_dataset"
d_set = pd.read_csv('./../../Data/'+dataset_name+'.csv',nrows=16)
d_set['result']=d_set['result'].astype('int')
dset_features = d_set.copy()
dset_labels = dset_features.pop('result')
dset_features = np.array(dset_features)

# Reshape features to include timestep dimension
# Assuming each row in the CSV file represents a timestep
# Reshape to (number_of_samples, timesteps, number_of_features)
timesteps = 1  # Assuming each row represents a timestep
dset_features_reshaped = dset_features.reshape(-1, timesteps, dset_features.shape[1])

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(dset_features, dset_labels, test_size=0.2, random_state=42)

# Configure the model with multiple layers
model = tf.keras.Sequential([
            layers.LSTM(32, activation='relu', input_shape=(dset_features.shape[1],1)),
            layers.Dense(64, activation='relu'),
            layers.Dense(1, activation='linear')  # Output layer with linear activation for regression
            ])
model.compile(optimizer="adam", loss="mean_squared_error", metrics=['mae'])

# Train the model
model, history, predictions, training_time = train_model(model, X_train, y_train, X_test, y_test,model_name='LSTM',dataset_name=dataset_name,total_epoch=50)

# Test the model
test_metrics = test_model(model, X_test, y_test,model_name='LSTM',dataset_name=dataset_name)

Epoch 1/50


c:\Users\akars\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - loss: 546036612727832576.0000 - mae: 516242784.0000 - val_loss: 148205629029220352.0000 - val_mae: 192491328.0000
Epoch 2/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step - loss: 546036612727832576.0000 - mae: 516242784.0000 - val_loss: 148205629029220352.0000 - val_mae: 192491328.0000
Epoch 3/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step - loss: 546036612727832576.0000 - mae: 516242784.0000 - val_loss: 148205629029220352.0000 - val_mae: 192491328.0000
Epoch 4/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step - loss: 546036612727832576.0000 - mae: 516242784.0000 - val_loss: 148205629029220352.0000 - val_mae: 192491328.0000
Epoch 5/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step - loss: 546036612727832576.0000 - mae: 516242784.0000 - val_loss: 148205629029220352.0000 - val_mae: 192491328.0000
Epoch 6/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step - loss: 546036612727832576.0000 - mae: 516242784.0000 - val_loss: 148205629029220352.0000 - val_mae: 192491328.0000
Epoch 7/50
1/1 ━━━━━━━━

In [18]:
# Load dataset
dataset_name = "natural_log_dataset"
d_set = pd.read_csv('./../../Data/'+dataset_name+'.csv')
dset_features = d_set.copy()
dset_labels = dset_features.pop('result')
dset_features = np.array(dset_features)

# Reshape features to include timestep dimension
# Assuming each row in the CSV file represents a timestep
# Reshape to (number_of_samples, timesteps, number_of_features)
timesteps = 1  # Assuming each row represents a timestep
dset_features_reshaped = dset_features.reshape(-1, timesteps, dset_features.shape[1])

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(dset_features, dset_labels, test_size=0.2, random_state=42)

# Configure the model with multiple layers
model = tf.keras.Sequential([
            layers.LSTM(32, activation='relu', input_shape=(dset_features.shape[1],1)),
            layers.Dense(64, activation='relu'),
            layers.Dense(1, activation='linear')  # Output layer with linear activation for regression
            ])
model.compile(optimizer="adam", loss="mean_squared_error", metrics=['mae'])

# Train the model
model, history, predictions, training_time = train_model(model, X_train, y_train, X_test, y_test,model_name='LSTM',dataset_name=dataset_name,total_epoch=1500,ea=True,eamonitor="mae")

# Test the model
test_metrics = test_model(model, X_test, y_test,model_name='LSTM',dataset_name=dataset_name)

Epoch 1/1500


c:\Users\akars\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


500/500 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 269.4160 - mae: 5.1834 - val_loss: 11.3664 - val_mae: 2.8909
Epoch 2/1500
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 10.7314 - mae: 2.8047 - val_loss: 9.1119 - val_mae: 2.5822
Epoch 3/1500
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 8.4553 - mae: 2.4974 - val_loss: 6.6592 - val_mae: 2.2145
Epoch 4/1500
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 6.0497 - mae: 2.1227 - val_loss: 4.4760 - val_mae: 1.8193
Epoch 5/1500
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 3.9177 - mae: 1.7060 - val_loss: 2.6695 - val_mae: 1.4176
Epoch 6/1500
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 2.3207 - mae: 1.3237 - val_loss: 1.4022 - val_mae: 1.0434
Epoch 7/1500
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 1.1326 - mae: 0.9306 - val_loss: 0.5854 - val_mae: 0.6796
Epoch 8/1500
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.4632 - mae: 0.5993 - val_loss: 0.2125 - val_mae: 0.4093
Epoch 9/1500
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/